In [9]:
# Missile_Environment for Reinforcement Learning
#    v-1.6
#       LPF available
#       Normalize all state

import vpython as vp

import math as m
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import time
import copy
import Missile_ENV_V_3_Ldot_Vm_ideal as ENV

rad2deg         = 57.29577951
deg2rad         = 0.01745329252
gpu_num         = 0

In [10]:
device      = ('cuda'+':'+ str(gpu_num)) if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == ('cuda'+ ':' + str(gpu_num)):
    torch.cuda.manual_seed_all(777)

In [11]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        
        self.fc1     = nn.Linear(4, 150, bias = True)    
        self.relu1   = nn.ReLU()
        self.fc2     = nn.Linear(150, 300, bias = True) 
        self.relu2   = nn.ReLU()
        self.fc3     = nn.Linear(300, 500, bias = True)
        self.relu3   = nn.ReLU()
        self.fc4     = nn.Linear(500, 300, bias = True)
        self.relu4   = nn.ReLU()
        self.fc5     = nn.Linear(300, 150, bias = True)
        self.relu5   = nn.ReLU()
        self.fc6     = nn.Linear(150, 1, bias = True)
        
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        torch.nn.init.xavier_uniform_(self.fc3.weight)
        torch.nn.init.xavier_uniform_(self.fc4.weight)
        torch.nn.init.xavier_uniform_(self.fc5.weight)
        torch.nn.init.xavier_uniform_(self.fc6.weight)
        
    def forward(self, x):
        
        out = self.fc1(x)
        out = self.relu1(out)
        
        out = self.fc2(out)
        out = self.relu2(out)
        
        out = self.fc3(out)
        out = self.relu3(out)
        
        out = self.fc4(out)
        out = self.relu4(out)
        
        out = self.fc5(out)
        out = self.relu5(out)
        
        out = self.fc6(out)
        
        return out

In [17]:
def ITCG(Vm, R, lam_psi, N, d_lam, s, k1):
    acp1 = (-Vm*(lam_psi)*m.cos(lam_psi))/2/R
    acp2 = (Vm*(2*N-1)*(1-m.cos(lam_psi)))/R/(lam_psi)
    sterm = k1*lam_psi*s
    a_c = d_lam+acp1+acp2+sterm
    aM = a_c*Vm
    return aM

def get_look(seeker):
    return seeker.LOS

def get_dlam(Vm,R,look):
    return Vm/R*m.sin(look)

def get_analytic_T2go(look, N, Rmag, Vm):
    return (1+((look)**2)/2/(2*N-1))*Rmag/Vm

def get_neural_T2go(model):
    return 'asdfasdf'

def get_s(t, t_d, t2go):
    return t - t_d + t2go

def get_k():
    return 1


In [12]:
model = DNN().to(device)
model.load_state_dict(torch.load('./WEIGHT_8.2560e-07_notquite', map_location=torch.device('cpu')))
model.eval()
model.train(False)


DNN(
  (fc1): Linear(in_features=4, out_features=150, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=150, out_features=300, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=300, out_features=500, bias=True)
  (relu3): ReLU()
  (fc4): Linear(in_features=500, out_features=300, bias=True)
  (relu4): ReLU()
  (fc5): Linear(in_features=300, out_features=150, bias=True)
  (relu5): ReLU()
  (fc6): Linear(in_features=150, out_features=1, bias=True)
)

In [13]:
DATmean     = np.array([ 5.0023467e+03,-1.5984995e+00,1.1228751e+02,1.7544924e+00,2.1465954e+01],dtype='float32')
STDmean     = np.array([ 5.5374233e+03, 1.7694662e+00,1.6407622e+02,1.4172097e+00,2.4340271e+01],dtype='float32')

In [16]:
initpos    = [5000., 3200.]
scavel       = 250.
initpsi     = deg2rad*150
dt          = 0.01
Missile1    = ENV.Missile_2D(scavel, initpos[0], initpos[1], initpsi, dt)
TargetShip  = ENV.TargetShip(0., 0., 0., 0.)
Missile1EYE = ENV.Seeker(Missile1, TargetShip)

In [ ]:
mode = 'n' #stands for neural
mode = 'a' #stands for analytic
with torch.no_grad():
    
    if mode == 'a':
        while true:
            acc = ITCG( Missile1.scavel,\
                        MissileEYE.R,\
                        get_look(MissileEYE),\
                        N,\
                        get_dlam(Missile1.scavel, MissileEYE.R, lookget_look(MissileEYE)),\
                        get_s(t, t_d, t2go),\
                        1)

In [ ]:
noise_deviation = 0.1

testFilename = '/home/workstation2/Daseon/DeepMonopulse/datTest_noNoise_Table2_10000.csv'
ff = open(testFilename,'r')
Reader = csv.reader(ff)
testDataset_raw = []
for row in Reader:
    testDataset_raw.append(row)
testDataset_raw = np.array(testDataset_raw, dtype='float32')
ff.close()

constpart  = copy.deepcopy(testDataset_raw[:, :2])
signpart   = copy.deepcopy(testDataset_raw[:, 2:-2])
anspart    = copy.deepcopy(testDataset_raw[:, -2:])

noise = copy.deepcopy(np.random.normal(0, noise_deviation, (testDataset_raw.shape[0],4)))

dirtysig   = np.array(signpart+noise,dtype='float32')
testDataset_regen = copy.deepcopy( np.concatenate((constpart, dirtysig, anspart), axis=1) )

print(testDataset_regen[:10,:])

testDataset = np.array((testDataset_regen - trainMean)/trainStd, dtype='float32')
    

In [ ]:
with torch.no_grad():
    datalenth = 10000
    index = np.random.choice(testDataset.shape[0], datalenth, replace=False)
    
    inputt = testDataset[index, :-2]
    
    inference = model(torch.from_numpy(inputt).to(device))
    answer = testDataset_raw[index, -2:]
    
    inference = inference.to('cpu')
    
    dummy_input = torch.from_numpy(testDataset[0, :-2]).to(device)
    
    
    
    predicVal = (inference*trainStd[-2:]+trainMean[-2:])
    predicVal = predicVal.numpy()*rad2deg
    answerVal = answer*rad2deg
    print('predic : ',predicVal)
    print('Realvl : ',answerVal)
    
    print('\n\n',np.sqrt(np.sum((predicVal-answerVal)**2, axis=0)/predicVal.shape[0]))

In [ ]:
testDataset[index, :-2]
print(noise.shape)
print(inputt.shape)
print((noise+inputt).shape)

In [ ]:
kkk = [[1,1,3],[3,8,5],[6,7,7],[8,6,2],[8,1,1]]
kkk = np.array(kkk)

In [ ]:
kkk.random.choice(5,3)

In [ ]:
kkk[np.array([1,3]),0:-2]

In [ ]:
mm = np.mean(kkk, axis=0)
mm

In [ ]:
ss = np.std(kkk, axis=0)
ss

In [ ]:
uu = np.sum(kkk, axis=0)
uu

In [ ]:
inference

In [ ]:
torch.onnx.export(model,               # 실행될 모델
                dummy_input,                         # 모델 입력값 (튜플 또는 여러 입력값들도 가능)
                "3.e-6.onnx",   # 모델 저장 경로 (파일 또는 파일과 유사한 객체 모두 가능)
                export_params=True,        # 모델 파일 안에 학습된 모델 가중치를 저장할지의 여부
                opset_version=10,          # 모델을 변환할 때 사용할 ONNX 버전
                input_names = ['input'],   # 모델의 입력값을 가리키는 이름
                output_names = ['output'] # 모델의 출력값을 가리키는 이름
                  )

In [ ]:
#torch.save(model.state_dict(), './WEIGHT_'+str(avg_cost.item()))

In [ ]:
print(avg_cost.item())

In [ ]:
a= np.array([[1,2],[2,2],[3,8]],dtype='float32')
b= np.array([[5,1],[1,3],[2,2]])

In [ ]:
(a-b)

In [ ]:
(a-b)**2

In [ ]:
np.sum((a-b)**2, axis=0)

In [ ]:
np.sum((a-b)**2, axis=0)/a.shape[0]

In [ ]:
np.sqrt(np.sum((a-b)**2, axis=0)/a.shape[0])

In [ ]:
a+b

In [ ]:
np.concatenate((a,b,b), axis=1)